In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pynq import allocate
import xrfdc
import time

from pynq.overlays.marcus_base import marcus_BaseOverlay

base = marcus_BaseOverlay('marcus_base.bit')

base.init_rf_clks()

In [2]:
import socket

# the pl sends 45124 / 2 packets (sampling rate halved via hilbert) of 2 values each which have real and complex part which are 32 bit long each but python doesnt know the difference
# the 4*32 bits are packed into a 128 bit bus
num_samples = int(13852)
num_values = int(num_samples*2)
data_type = np.int32

# hardware_avgs*software_traces should be a multiple of 20000
hardware_avgs = 10000
software_traces = 40

buffer_trig = allocate(shape=(num_values,), dtype=data_type)
buffer_log = allocate(shape=(20000*4,), dtype=np.float32)

base.radio.avgs_config_writer.write(base.radio.avgs_config_writer.register_map.send.address, hardware_avgs)
base.radio.passer_config_writer.write(base.radio.passer_config_writer.register_map.send.address, 1)
base.radio.axi_dma_log.recvchannel.transfer(buffer_log)
base.radio.axi_dma_avg.recvchannel.transfer(buffer_trig)

a = time.time()
avgs_current = 0
for idx in np.arange(software_traces):
    base.radio.axi_dma_avg.recvchannel.wait()
    data = np.copy(np.array(buffer_trig))
    base.radio.axi_dma_avg.recvchannel.transfer(buffer_trig)

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect(("192.168.3.145", 1024))    
        sock.send(data.tobytes())
    print(idx)
    del(data)
    
    avgs_current = avgs_current + hardware_avgs
    if avgs_current>=20000:
        base.radio.axi_dma_log.recvchannel.wait()
        log_data = np.copy(np.array(buffer_log))
        base.radio.axi_dma_log.recvchannel.transfer(buffer_log)
        avgs_current = 0
        print('read phase log')
        del(log_data)
    print(time.time()-a)

base.radio.passer_config_writer.write(base.radio.passer_config_writer.register_map.send.address, 0)


0
0.530947208404541
1
read phase log
1.0420258045196533
2
1.5543279647827148
3
read phase log
2.0711522102355957
4
2.582239866256714
5
read phase log
3.0978288650512695
6
3.610750436782837
7
read phase log
4.1212592124938965
8
4.63463020324707
9
read phase log
5.145615339279175
10
5.658950090408325
11
read phase log
6.169417381286621
12
6.683265686035156
13
read phase log
7.19364333152771
14
7.706523656845093
15
read phase log
8.217469692230225
16
8.731110572814941
17
read phase log
9.242005586624146
18
9.755064487457275
19
read phase log
10.265210628509521
20
10.778828620910645
21
read phase log
11.289391994476318
22
11.803038358688354
23
read phase log
12.314013719558716
24
12.826658487319946
25
read phase log
13.33730673789978
26
13.847816944122314
27
read phase log
14.3614182472229
28
14.87535834312439
29
read phase log
15.385388851165771
30
15.898637771606445
31
read phase log
16.40987777709961
32
16.92315173149109
33
read phase log
17.433276176452637
34
17.946890354156494
35
read